<a href="https://colab.research.google.com/github/hercules2004/OOAD_new/blob/master/text_to_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchaudio librosa coqui-tts datasets jiwer rouge-score


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 66.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 64.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 82.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return GPU name


True
Tesla T4


In [ ]:
from TTS.api import TTS

# Load XTTS v2 (multi-accent, multi-gender support)
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y


100%|█████████▉| 1.87G/1.87G [00:44<00:00, 42.1MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 42.1MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 34.8kiB/s]
 87%|████████▋ | 313k/361k [00:00<00:00, 2.52MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 1.39MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 143iB/s]
100%|██████████| 7.75M/7.75M [00:15<00:00, 29.9MiB/s]

In [ ]:
from google.colab import drive
import os
import zipfile
import torchaudio
import torchaudio.transforms as T
import torch
import re

# ===============================
# 📌 STEP 1: MOUNT GOOGLE DRIVE
# ===============================
drive.mount('/content/drive')

# ===============================
# 📌 STEP 2: EXTRACT VCTK ZIP FILE
# ===============================
# Path to ZIP file in Google Drive
drive_dataset_path = "/content/drive/MyDrive/VCTK/vctk.zip"  # Update path if needed
extract_path = "/content/VCTK"  # Where the dataset will be extracted

# Ensure the target directory exists
os.makedirs(extract_path, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(drive_dataset_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

# ===============================
# 📌 STEP 3: DEFINE FILE PATHS
# ===============================
TARGET_SR = 22050  # Target sampling rate
audio_folder = os.path.join(extract_path, "wav48")  # Update based on extracted folder structure
text_folder = os.path.join(extract_path, "txt")

# ===============================
# 📌 STEP 4: FUNCTION TO CLEAN TEXT
# ===============================
def clean_text(text):
    """Remove unwanted characters and normalize text."""
    text = text.lower().strip()  # Convert to lowercase and remove leading/trailing spaces
    text = re.sub(r"[^a-zA-Z0-9\s.,!?']", "", text)  # Keep only alphanumeric and punctuation
    return text

# ===============================
# 📌 STEP 5: FUNCTION TO PROCESS AUDIO AND TEXT
# ===============================
def process_data(audio_filename):
    try:
        # Load and clean text
        text_file = os.path.join(text_folder, os.path.splitext(audio_filename)[0] + ".txt")
        with open(text_file, "r") as f:
            text = clean_text(f.read().strip())

        # Load audio
        audio_path = os.path.join(audio_folder, audio_filename)
        audio, sr = torchaudio.load(audio_path)

        # Resample if needed
        if sr != TARGET_SR:
            resampler = T.Resample(orig_freq=sr, new_freq=TARGET_SR)
            audio = resampler(audio)

        # Convert stereo to mono
        if audio.shape[0] > 1:
            audio = torch.mean(audio, dim=0, keepdim=True)

        # Normalize audio
        audio = audio / torch.max(torch.abs(audio))

        # Trim silence
        trimmed_audio = T.Vad(sample_rate=TARGET_SR)(audio)

        # Speaker ID (Extracted from filename format)
        speaker_id = audio_filename.split("_")[0]  # Adjust based on dataset naming convention

        return {"text": text, "audio": trimmed_audio, "speaker_id": speaker_id}

    except Exception as e:
        print(f"⚠️ Error processing {audio_filename}: {e}")
        return None  # Skip corrupt data

# ===============================
# 📌 STEP 6: PROCESS ENTIRE DATASET
# ===============================
# List all audio files
audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])

# Process each file
processed_dataset = [process_data(f) for f in audio_files]
processed_dataset = [data for data in processed_dataset if data is not None]  # Remove failed samples

print(f"✅ Processed {len(processed_dataset)} audio samples successfully!")


FSTimeoutError: 

In [ ]:
from torch.utils.data import DataLoader
import torchaudio.transforms as T
import torch
import torch.nn.functional as F
import numpy as np

# ================================
# 📌 STEP 1: DEFINE MEL-SPECTROGRAM LOSS
# ================================
mel_transform = T.MelSpectrogram(sample_rate=22050, n_mels=80)

def mel_loss(pred_audio, target_audio):
    pred_mel = mel_transform(pred_audio)
    target_mel = mel_transform(target_audio)
    return F.mse_loss(pred_mel, target_mel)

# ================================
# 📌 STEP 2: OPTIMIZER + MIXED PRECISION TRAINING
# ================================
optimizer = torch.optim.Adam(tts.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()  # Mixed Precision Training

# ================================
# 📌 STEP 3: CACHING SPEAKER EMBEDDINGS
# ================================
speaker_embeddings = {}
def get_speaker_embedding(speaker_id):
    if speaker_id not in speaker_embeddings:
        ref_audio_path = f"speaker_refs/{speaker_id}.wav"
        speaker_embeddings[speaker_id] = tts.get_speaker_embedding(ref_audio_path)
    return speaker_embeddings[speaker_id]

# ================================
# 📌 STEP 4: DATALOADER WITH PADDING
# ================================
def collate_fn(batch):
    texts = [b["text"] for b in batch]
    speaker_ids = [b["speaker_id"] for b in batch]

    # Load raw audio and determine max length for padding
    audio_tensors = [b["audio"].squeeze(0) for b in batch]
    max_length = max([a.shape[-1] for a in audio_tensors])

    # Pad audios to the same length
    padded_audios = torch.zeros(len(batch), max_length).to("cuda")
    for i, audio in enumerate(audio_tensors):
        padded_audios[i, :audio.shape[-1]] = audio  # Pad shorter sequences

    return texts, padded_audios, speaker_ids

dataloader = DataLoader(processed_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# ================================
# 📌 STEP 5: TRAINING LOOP (WITH MIXED PRECISION)
# ================================
for epoch in range(10):
    total_loss = 0
    optimizer.zero_grad()

    for i, (texts, audios, speaker_ids) in enumerate(dataloader):
        speaker_embs = [get_speaker_embedding(sid) for sid in speaker_ids]

        # Generate speech
        output_audios = []
        for text, emb in zip(texts, speaker_embs):
            synthesized_audio = tts.synthesize(text, speaker_embedding=emb)
            synthesized_audio = torch.from_numpy(np.array(synthesized_audio)).float().to("cuda")
            output_audios.append(synthesized_audio)

        # Pad generated audio to the same length as target audio
        max_length = audios.shape[-1]
        output_padded = torch.zeros(len(output_audios), max_length).to("cuda")
        for i, audio in enumerate(output_audios):
            output_padded[i, :audio.shape[-1]] = audio  # Pad shorter sequences

        # Compute loss with mixed precision
        with torch.cuda.amp.autocast():
            loss = mel_loss(output_padded, audios)

        scaler.scale(loss).backward()

        # Gradient accumulation (update every 4 batches)
        if (i + 1) % 4 == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()

        # Free memory
        del output_padded, audios, output_audios
        torch.cuda.empty_cache()

    print(f"Epoch {epoch + 1}: Avg Loss = {total_loss / len(dataloader)}")


In [ ]:
from jiwer import wer
from rouge_score import rouge_scorer
import speech_recognition as sr
import librosa
import io
import soundfile as sf
import torch
import whisper
import re

# Load Whisper ASR Model
whisper_model = whisper.load_model("base")  # Change to "small" or "large" for better accuracy

# Text normalization function
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove punctuation
    return text

def evaluate_tts(model, text, reference_audio):
    """Evaluates a TTS model by comparing synthesized speech with original text."""

    # Generate speech
    generated_audio = model.synthesize(text, speaker_wav=reference_audio)

    # Convert audio to an in-memory file (avoid saving to disk)
    buffer = io.BytesIO()
    sf.write(buffer, generated_audio, 22050, format="WAV")
    buffer.seek(0)

    # Load the audio for validation
    audio_data, _ = librosa.load(buffer, sr=22050)

    # Check if audio is valid (non-empty, not just silence)
    if len(audio_data) < 5000:  # Adjust threshold as needed
        print("Generated audio is too short or silent.")
        return {"Error": "Invalid audio output"}

    # Automatic Speech Recognition (ASR) - Use Whisper if available
    try:
        buffer.seek(0)  # Reset buffer position
        transcription = whisper_model.transcribe(buffer)["text"]
    except Exception:
        print("Whisper ASR failed, falling back to Google ASR.")
        recognizer = sr.Recognizer()
        with sr.AudioFile(buffer) as source:
            audio = recognizer.record(source)
            try:
                transcription = recognizer.recognize_google(audio)
            except sr.UnknownValueError:
                print("ASR could not understand the audio.")
                transcription = ""
            except sr.RequestError:
                print("ASR request failed. Check your internet connection.")
                transcription = ""

    # Normalize text before evaluation
    original_text = normalize_text(text)
    generated_text = normalize_text(transcription)

    # Compute ROUGE scores
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(original_text, generated_text)

    # Compute WER
    wer_score = wer(original_text, generated_text)

    # Display results
    results = {
        "ROUGE-1": scores["rouge1"].fmeasure,
        "ROUGE-2": scores["rouge2"].fmeasure,
        "ROUGE-L": scores["rougeL"].fmeasure,
        "WER": wer_score
    }
    print(f"Evaluation Results: {results}")

    return results

# Example Evaluation
text_input = "Hello, how are you today?"
evaluate_tts(tts, text_input, "reference_voice.wav")


In [ ]:
import os

# Ensure fine-tuned model exists
if os.path.exists("fine_tuned_xtts.pth"):
    tts.load("fine_tuned_xtts.pth")
else:
    raise FileNotFoundError("Error: Fine-tuned model not found! Check the file path.")

# Ensure speaker reference file exists
reference_voice = "indian_male_reference.wav"
if not os.path.exists(reference_voice):
    print("Warning: Reference voice file not found! Using default voice.")
    reference_voice = None  # Use default model voice

# Get text input dynamically
text_input = input("Enter text to synthesize: ")

# Generate speech and save to file
tts.tts_to_file(text_input, speaker_wav=reference_voice, file_path="output.wav")

print("Speech synthesis complete! Output saved as output.wav")
